### MRMR-More

More experiments with the MRMR codebase.

In [1]:
import sys
sys.path.append("../..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import pathlib
import yaml
from wbf_simulate import run_nrobot1day


In [2]:
# Run an experiment with a different budget
experiment_path = pathlib.Path(Config()["experiment_external"])
experiment_path.mkdir(exist_ok=True)
experiment_data = pathlib.Path(Config()["data_external"])
experiment_data.mkdir(exist_ok=True)

Config().set_experiment_path(experiment_path)
Config().set_experiment_data(experiment_data)
# FIXME this is dangerous, because if it has the original paths, it overwrites the experiments, this needs to be fixed
Config().copy_experiment("mrmr")
Config().copy_experiment("environment")
Config().copy_experiment("policy")
Config().copy_experiment("estimator")
Config().copy_experiment("score")



***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\WaterBerryFarms\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\WBF-config-LotziYoga.yaml
***ExpRun**: Experiment config path changed to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external
***ExpRun**: Experiment data path changed to c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external
***ExpRun**: Experiment mrmr copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\mrmr
***ExpRun**: Experiment environment copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\environment
***ExpRun**: Experiment policy copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\policy
***ExpRun**: Experiment estimator copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\estimator
***ExpRun**: Experiment score copied to c:\Users\lboloni\Documents\Code\_TempData\WBF-exp-external\s

In [3]:
def generate_rw(n_robots = 1, budget = 1000, environment = "mrmr-custom-notclustered"):
    """Generate experiment with n robots following random waypoint"""
    runname = f"gen-rw-{n_robots}-b-{budget}"
    val = {}    
    val["name"] = "n random waypoint"

    val["exp_environment"] = "environment"
    val["run_environment"] = environment

    val["exp_estimator"] = "estimator"
    val["run_estimator"] = "adaptive-disk"

    val["exp_score"] = "score"
    val["run_score"] = "weighted_asymmetric"

    val["timesteps-per-day"] = budget
    val["time-start-environment"] = 19

    robots = []
    val["robots"] = robots

    for robot in range(n_robots):
        v = {}
        robots.append(v)
        v["name"] = f"robot-{robot}"
        v["exp-policy"] = "policy"
        v["run-policy"] = "random-waypoint"
        vextra = {}
        v["exp-policy-extra-parameters"] = vextra
        vextra["seed"] = robot
        vextra["policy-name"] = f"rwp-{robot}"
        vextra["budget"] = budget

    # save the generated path 
    path = pathlib.Path(Config().get_experiment_path(), "mrmr", runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    return runname

In [ ]:
def generate_lawnmowers(n_robots = 1, budget = 1000, environment = "mrmr-custom-notclustered"):
    """Generate experiment with n robots following lawnmower, with the area being divided into n areas"""
    runname = f"gen-lawn-{n_robots}-b-{budget}"
    val = {}    
    val["name"] = "n random waypoint"

    val["exp_environment"] = "environment"
    val["run_environment"] = environment

    val["exp_estimator"] = "estimator"
    val["run_estimator"] = "adaptive-disk"

    val["exp_score"] = "score"
    val["run_score"] = "weighted_asymmetric"

    val["timesteps-per-day"] = budget
    val["time-start-environment"] = 19

    robots = []
    val["robots"] = robots

    for robot in range(n_robots):
        v = {}
        robots.append(v)
        v["name"] = f"robot-{robot}"
        v["exp-policy"] = "policy"
        v["run-policy"] = "fixed-budget-lawnmower"
        vextra = {}
        v["exp-policy-extra-parameters"] = vextra
        area = [0,0,0,0]
        border = 5
        width = (100 - 2 * border) / n_robots
        area[0] = int(border)
        area[1] = int(border + robot * width)
        area[2] = 100 - border
        area[3] = int(border + (robot+1) * width)
        vextra["area"] = str(area)
        vextra["policy-name"] = f"lawn-{robot}"
        vextra["budget"] = budget

    # save the generated path 
    path = pathlib.Path(Config().get_experiment_path(), "mrmr", runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    return runname

In [ ]:
def generate_mrmr(n_robots = 1, budget = 1000, environment = "mrmr-custom-notclustered"):
    """Generate experiment with n robots following lawnmower, with the area being divided into n areas"""
    runname = f"gen-mrmr-{n_robots}-b-{budget}"
    val = {}    
    val["name"] = "n random waypoint"

    val["exp_environment"] = "environment"
    val["run_environment"] = environment

    val["exp_estimator"] = "estimator"
    val["run_estimator"] = "adaptive-disk"

    val["exp_score"] = "score"
    val["run_score"] = "weighted_asymmetric"

    val["timesteps-per-day"] = budget
    val["time-start-environment"] = 19

    robots = []
    val["robots"] = robots

    # generate the first robot, the pioneer
    v = {}
    robots.append(v)
    v["name"] = f"pio"
    v["exp-policy"] = "policy"
    v["run-policy"] = "generic"    
    vextra = {}
    v["exp-policy-extra-parameters"] = vextra
    vextra["policy-name"] = f"mrmr-pioneer"
    vextra["policy-code"] = "MRMR_Pioneer"
    vextra["seed"] = 1
    vextra["budget"] = budget

    # generate the other robots, the contractors
    for robot in range(n_robots-1):
        v = {}
        robots.append(v)
        v["name"] = f"con-{robot}"
        v["exp-policy"] = "policy"
        v["run-policy"] = "generic"
        vextra = {}
        v["exp-policy-extra-parameters"] = vextra
        vextra["seed"] = robot + 1
        vextra["policy-name"] = f"mrmr-contractor-{robot}"
        vextra["policy-code"] = "MRMR_Contractor"
        vextra["budget"] = budget

    # save the generated path 
    path = pathlib.Path(Config().get_experiment_path(), "mrmr", runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    return runname

In [ ]:
# run = generate_rw(n_robots=3, budget=1000)
# run = generate_lawnmowers(n_robots=3, budget=1000)
run = generate_mrmr(n_robots=3, budget=1000)

In [6]:
exp = Config().get_experiment("mrmr", run)
run_nrobot1day(exp)



***ExpRun**: Configuration for exp/run: mrmr/gen-rw-3-b-1000 successfully loaded
Results file already exists:
c:\Users\lboloni\Documents\Code\_TempData\WBF-data-external\mrmr\gen-rw-3-b-1000\results.pickle
Delete this file if re-running is desired.
